#### 붓꽃 품종 분류
- 목표 : 붗꽃의 3가지 품종을 분류하기
- 데이터셋 : 내장 데이터셋
- 피쳐 : 4개
- 타켓 : 품종 1개
- 학습 : 지도학습 > 분류

[1] 데이터 준비

In [47]:
# 모듈 로딩
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [48]:
# 내장 데이터셋 로딩
data = load_iris(as_frame=True)

In [49]:
# Bunch 인스턴스 => dict와 유사한 형태
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [50]:
featureDF = data['data']
targetSR = data['target']

In [51]:
featureDF.shape, targetSR.shape

((150, 4), (150,))

In [52]:
featureDF.head(1)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2


In [53]:
targetSR.head(1)

0    0
Name: target, dtype: int32

[2] 학습을 위한 데이터셋 준비 => 학습용, 검증용, 테스트용

In [54]:
# 학습용 & 테스트용 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetSR,stratify=targetSR)

In [55]:
# 학습용 & 검증용 분리
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train)

In [56]:
print(f'Train DS : {X_train.shape[0]}   {X_train.shape[0]/featureDF.shape[0]}%')
print(f'Val   DS : {X_val.shape[0]}   {X_val.shape[0]/featureDF.shape[0]:.2f}%')
print(f'Test  DS : {X_test.shape[0]}   {X_test.shape[0]/featureDF.shape[0]:.2f}%')

Train DS : 84   0.56%
Val   DS : 28   0.19%
Test  DS : 38   0.25%


[3] 교차검증 방식

In [57]:
# 모듈 로딩
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [58]:
# 모듈 인스턴스 생성
dtc_model = DecisionTreeClassifier()

In [59]:
# [3-1] K Fold 기반 -----------------------------------------
# 정확도 저장 리스트
accuracys = []

# KFold 인스턴스 생성 [기본 k=5]
kfold = KFold()

In [60]:
# K번만큼 K개 데이터셋으로 학습 진행
# -> K등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx, (train_index, val_index) in enumerate(kfold.split(featureDF),1):
    # print(f'train_index : {train_index.tolist()}')

    # X_train, X_val 데이터셋 설정
    X_train, y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val, y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    # 학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train, y_train)
    val_acc = dtc_model.score(X_val, y_val)
    
    accuracys.append([train_acc, val_acc])
    print(f'[{idx}번째] Train 정확도 : {train_acc}, Val 정확도 : {val_acc}')

[1번째] Train 정확도 : 1.0, Val 정확도 : 1.0
[2번째] Train 정확도 : 1.0, Val 정확도 : 1.0
[3번째] Train 정확도 : 1.0, Val 정확도 : 0.8333333333333334
[4번째] Train 정확도 : 1.0, Val 정확도 : 0.9333333333333333
[5번째] Train 정확도 : 1.0, Val 정확도 : 0.7666666666666667


In [61]:
accuracys

[[1.0, 1.0],
 [1.0, 1.0],
 [1.0, 0.8333333333333334],
 [1.0, 0.9333333333333333],
 [1.0, 0.7666666666666667]]

In [63]:
# 평균 계산
train_mean = sum([value[0] for value in accuracys])/kfold.n_splits
test_mean = sum([value[1] for value in accuracys])/kfold.n_splits

print(f'Train 정확도 : {train_mean}     Val 정확도 : {test_mean:.2f}')

Train 정확도 : 1.0     Val 정확도 : 0.91


In [64]:
# [3-2] StratifiedKFold : 정답/레이블/타겟의 비율을 고려해서 데이터 나눔

# 정확도 저장 리스트
accuracys = []

# StratifiedKFold 인스턴스 생성
skfold = StratifiedKFold()


In [66]:
# K번만큼 K개 데이터셋으로 학습 진행
# -> K등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx, (train_index, val_index) in enumerate(skfold.split(featureDF, targetSR),1):
    # print(f'train_index : {train_index.tolist()}')

    # X_train, X_val 데이터셋 설정
    X_train, y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val, y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    # 학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train, y_train)
    val_acc = dtc_model.score(X_val, y_val)
    
    accuracys.append([train_acc, val_acc])
    print(f'[{idx}번째] Train 정확도 : {train_acc}, Val 정확도 : {val_acc}')

[1번째] Train 정확도 : 1.0, Val 정확도 : 0.9666666666666667
[2번째] Train 정확도 : 1.0, Val 정확도 : 0.9666666666666667
[3번째] Train 정확도 : 1.0, Val 정확도 : 0.9
[4번째] Train 정확도 : 1.0, Val 정확도 : 0.9666666666666667
[5번째] Train 정확도 : 1.0, Val 정확도 : 1.0


In [67]:
# 평균 계산
train_mean = sum([value[0] for value in accuracys])/skfold.n_splits
test_mean = sum([value[1] for value in accuracys])/skfold.n_splits

print(f'Train 정확도 : {train_mean}     Val 정확도 : {test_mean:.2f}')

Train 정확도 : 1.0     Val 정확도 : 0.96


- 교차 검증 및 성능평가 동시 진행 함수 
    * => cross_val_score, cross_val_predict
    * => cross_validate

In [68]:
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate

In [92]:
#### [1] 전체 DS ==> 학습용과 테스트용 DS 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetSR, stratify= targetSR)

In [93]:
#### cross_val_predict
predict = cross_val_predict(dtc_model, X_train, y_train, cv=3)

In [94]:
print(f'predict : {predict}')

predict : [2 2 0 0 1 2 1 2 2 2 1 2 1 2 2 1 2 1 1 2 1 2 0 0 2 0 1 1 0 2 0 1 2 1 0 1 1
 0 2 0 2 2 0 1 2 0 1 0 0 2 0 1 0 0 1 0 1 1 1 1 1 2 0 2 1 1 1 2 2 1 1 1 2 2
 1 2 2 0 0 2 0 2 1 1 2 0 0 0 0 2 0 0 2 0 0 0 2 1 0 2 2 1 0 1 1 1 0 2 0 1 0
 0]


In [95]:
#### cross_val_score
cross_val_score(dtc_model, X_train, y_train, cv=3)

array([0.97368421, 0.91891892, 0.94594595])

In [104]:
#### cross_validate
result = cross_validate(dtc_model, X_train, y_train, cv=10,
                        return_train_score=True,
                        return_estimator=True)

In [105]:
result

{'fit_time': array([0.00413084, 0.00199485, 0.00197744, 0.00150657, 0.00099397,
        0.00199986, 0.00099993, 0.00199914, 0.00199962, 0.00099945]),
 'score_time': array([0.00200081, 0.00201416, 0.00200295, 0.00100493, 0.00099993,
        0.00099993, 0.00099969, 0.00099945, 0.00099993, 0.00100017]),
 'estimator': [DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier()],
 'test_score': array([0.83333333, 1.        , 1.        , 1.        , 0.72727273,
        1.        , 1.        , 1.        , 0.90909091, 1.        ]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [106]:
resultDF = pd.DataFrame(result)[['test_score', 'train_score']]
resultDF

,test_score,train_score
0,0.833333,1.0
1,1.000000,1.0
2,1.000000,1.0
3,1.000000,1.0
4,0.727273,1.0
5,1.000000,1.0
6,1.000000,1.0
7,1.000000,1.0
8,0.909091,1.0
9,1.000000,1.0


- 최적화된 모델 추출

In [107]:
best_model = result['estimator'][1]
best_model

DecisionTreeClassifier()

- 테스트 데이터로 확인

In [108]:
best_model.predict(X_test)

array([1, 0, 1, 0, 1, 2, 2, 2, 0, 1, 1, 0, 1, 0, 0, 2, 1, 1, 2, 0, 0, 2,
       0, 2, 0, 0, 2, 1, 2, 1, 2, 1, 2, 0, 1, 2, 0, 2])

In [109]:
best_model.score(X_test, y_test)

0.9473684210526315